In [26]:
#@title MIT License
#
# Copyright (c) 2017 François Chollet
#
# Permission is hereby granted, free of charge, to any person obtaining a
# copy of this software and associated documentation files (the "Software"),
# to deal in the Software without restriction, including without limitation
# the rights to use, copy, modify, merge, publish, distribute, sublicense,
# and/or sell copies of the Software, and to permit persons to whom the
# Software is furnished to do so, subject to the following conditions:
#
# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.
#
# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL
# THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING
# FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER
# DEALINGS IN THE SOFTWARE.

In [1]:
from __future__ import absolute_import, division,print_function,\
    unicode_literals

In [2]:
import tensorflow as tf

In [3]:
tf.enable_eager_execution()

In [4]:
import numpy as np
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("hub version: ", hub.__version__)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Version:  1.14.0
Eager mode:  True
hub version:  0.6.0


In [5]:
# Download IMDB dataset
# Split training set 60%/40%
train_validation_split = tfds.Split.TRAIN.subsplit([6,4])

In [6]:
(train_data, validation_data), test_data = tfds.load(\
    name="imdb_reviews",\
    split=(train_validation_split, tfds.Split.TEST), \
    as_supervised=True)

In [7]:
# Explore data
# Print first 10
train_examples_batch, train_labels_batch = \
    next(iter(train_data.batch(10)))
train_examples_batch

<tf.Tensor: id=220, shape=(10,), dtype=string, numpy=
array([b"As a lifelong fan of Dickens, I have invariably been disappointed by adaptations of his novels.<br /><br />Although his works presented an extremely accurate re-telling of human life at every level in Victorian Britain, throughout them all was a pervasive thread of humour that could be both playful or sarcastic as the narrative dictated. In a way, he was a literary caricaturist and cartoonist. He could be serious and hilarious in the same sentence. He pricked pride, lampooned arrogance, celebrated modesty, and empathised with loneliness and poverty. It may be a clich\xc3\xa9, but he was a people's writer.<br /><br />And it is the comedy that is so often missing from his interpretations. At the time of writing, Oliver Twist is being dramatised in serial form on BBC television. All of the misery and cruelty is their, but non of the humour, irony, and savage lampoonery. The result is just a dark, dismal experience: the story p

In [8]:
train_labels_batch

<tf.Tensor: id=221, shape=(10,), dtype=int64, numpy=array([1, 1, 1, 1, 1, 1, 0, 1, 1, 0])>

In [12]:
# Build layers
# Use pretrained text embedding model from TF Hub
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

In [13]:
hub_layer = hub.KerasLayer(embedding, input_shape=[],
                          dtype=tf.string, trainable=True)

In [14]:
hub_layer(train_examples_batch[:3])

<tf.Tensor: id=402, shape=(3, 20), dtype=float32, numpy=
array([[ 3.9819887 , -4.4838037 ,  5.177359  , -2.3643482 , -3.2938678 ,
        -3.5364532 , -2.4786978 ,  2.5525482 ,  6.688532  , -2.3076782 ,
        -1.9807833 ,  1.1315885 , -3.0339816 , -0.7604128 , -5.743445  ,
         3.4242578 ,  4.790099  , -4.03061   , -5.992149  , -1.7297493 ],
       [ 3.4232912 , -4.230874  ,  4.1488533 , -0.29553518, -6.802391  ,
        -2.5163853 , -4.4002395 ,  1.905792  ,  4.7512794 , -0.40538004,
        -4.3401685 ,  1.0361497 ,  0.9744097 ,  0.71507156, -6.2657013 ,
         0.16533905,  4.560262  , -1.3106939 , -3.1121316 , -2.1338716 ],
       [ 3.8508697 , -5.003031  ,  4.8700504 , -0.04324996, -5.893603  ,
        -5.2983093 , -4.004676  ,  4.1236343 ,  6.267754  ,  0.11632943,
        -3.5934832 ,  0.8023905 ,  0.56146765,  0.9192484 , -7.3066816 ,
         2.8202746 ,  6.2000837 , -3.5709393 , -4.564525  , -2.305622  ]],
      dtype=float32)>

In [21]:
# Build model
model = tf.keras.Sequential()
# Uses pretrained saved model to map sentence into its embedding
# vector
model.add(hub_layer)
# FC layer
model.add(tf.keras.layers.Dense(16, activation='relu'))
# Single output node. Sigmoid -> [0,1]
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy',\
             metrics=['accuracy'])

In [23]:
# Train model
history = model.fit(train_data.shuffle(10000).batch(512),\
                   epochs=20, \
                   validation_data=validation_data.batch(512),\
                   verbose=1)

Epoch 1/20
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


30/30 [==============================] - 33s 1s/step - loss: 0.7095 - acc: 0.5843 - val_loss: 0.0000e+00 - val_acc: 0.0000e+00
Epoch 2/20
30/30 [==============================] - 21s 697ms/step - loss: 0.5957 - acc: 0.6875 - val_loss: 0.5795 - val_acc: 0.7012
Epoch 3/20
30/30 [==============================] - 19s 635ms/step - loss: 0.5581 - acc: 0.7179 - val_loss: 0.5474 - val_acc: 0.7353
Epoch 4/20
30/30 [==============================] - 27s 899ms/step - loss: 0.5238 - acc: 0.7516 - val_loss: 0.5179 - val_acc: 0.7595
Epoch 5/20
30/30 [==============================] - 21s 690ms/step - loss: 0.4902 - acc: 0.7785 - val_loss: 0.4892 - val_acc: 0.7756
Epoch 6/20
30/30 [==============================] - 21s 698ms/step - loss: 0.4558 - acc: 0.8019 - val_loss: 0.4607 - val_acc: 0.7923
Epoch 7/20
30/30 [==============================] - 19s 643ms/step - loss: 0.4213 - acc: 0.8228 - val_loss: 0.4330 - val_acc: 0.8083
Epoch 8/20
30/30 [==============================] - 18s 598ms/step - loss: 

In [24]:
# Evaluate model
results = model.evaluate(test_data.batch(512), verbose=2)

In [25]:
for name, val in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, val))

loss: 0.320
acc: 0.862
